# Gather Sentences from SEC Edgar
### **Purpose**: To gather data from the SEC website. Convert either an individual filing, or an RSS feed of filings, into a database of sentences (both before processing and after processing). Data can later be used to compare across periods. 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from Helpers import load, textpreprocess, cosinesimilarity
import pandas as pd
import sqlite3
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set Up

In [15]:
ticker = "ZG"
zg_rss_feed = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001617640&type=10&dateb=&owner=exclude&count=5&output=atom"
zg_10k_link = "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001617640/000161764021000012/z-20201231.htm"
database_name = "./Data/sentences.db"

# Load Individual

In [4]:
filing = textpreprocess.turn_filing_into_sentences_df(zg_10k_link)

In [10]:
filing.sample(5, random_state=1)

,Dirty,Clean
1294,We tested business process controls to reconci...,tested business process control reconcile vari...
107,"To date, Zillow Offers makes up less than 0.1%...",date zillow offer make le housing transaction...
1225,If any of the assumptions used in the Black-Sc...,assumption used blackscholesmerton model chang...
2246,Amended and Restated 2011 Incentive Plan (File...,amended restated incentive plan filed exhibit ...
1053,We recorded an income tax benefit of $7.5 mill...,recorded income tax benefit million year ende...


In [ ]:
# load.append_to_database(
#     data=filing,
#     database_name=database_name,
#     table_name=ticker,
#     year=2020,
#     period="Q4",
#     link=zg_10k_link
# )

# Load From RSS Feed

In [19]:
load.populate_database_from_rss_feed(database_name=database_name, table_name=ticker, rss_feed=zg_rss_feed, ticker=ticker)

Entry already exists in DB.
Loaded Q2-2021 to the database!
Loaded Q1-2021 to the database!
Loaded Q4-2020 to the database!
Loaded Q3-2020 to the database!
Loaded Q2-2020 to the database!
Loaded Q1-2020 to the database!
Loaded Q4-2019 to the database!
Loaded Q3-2019 to the database!
Loaded Q2-2019 to the database!
Loaded Q1-2019 to the database!


# Explore

In [20]:
engine = sqlite3.connect(database_name)

In [21]:
df = pd.read_sql(f"Select * from {ticker}", engine).drop(columns=["index", "Link"])
df.groupby(["Year", "Period"]).count()
engine.close()

Dirty  Clean  Type  Ticker
Year Period                            
2019 Q1        682    682   682     682
     Q2        743    743   743     743
     Q3        885    885   885     885
     Q4       2171   2171  2171    2171
2020 Q1        892    892   892     892
     Q2        914    914   914     914
     Q3        845    845   845     845
     Q4       2266   2266  2266    2266
2021 Q1        639    639   639     639
     Q2        717    717   717     717

In [23]:
df.head()

,Dirty,Clean,Year,Period,Type,Ticker
0,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,united state security exchange commission wash...,2021,Q2,10Q,ZG
1,Yes No Indicate by check mark whether the regi...,yes indicate check mark whether registrant ha ...,2021,Q2,10Q,ZG
2,Yes No Indicate by check mark whether the regi...,yes indicate check mark whether registrant lar...,2021,Q2,10Q,ZG
3,See the definitions of “large accelerated file...,see definition “large accelerated filer” “acce...,2021,Q2,10Q,ZG
4,"If an emerging growth company, indicate by che...",emerging growth company indicate check mark re...,2021,Q2,10Q,ZG


In [25]:
recent_annual_report = df.query(" Year == '2021' and Period == 'Q2' ")
last_annual_report = df.query(" Year == '2020' and Period == 'Q2' ")

# determine the outlier threshold for sentence length
words_in_sentence_threshold = recent_annual_report.Dirty.str.split(" ").str.len().quantile(.99) 

In [26]:
stop_words = ["digital", "company", "group", "key", "inc", "equity", "consolidated", "digitalbridge", "investment", "currently", "asset", "total", '"', " ", "interest", "value",
              "company's", "net", "income", "operating", "fair", "june", "million", "note", "operation", "stock", "management", "debt", "upon", "common", "business", "share",
               "cash", "cost", "held", "rate", "ended", "—", "month", "including", "real", "liability", "table", "financial", "may", "amount", "credit", "certain", "related", 
              "estate", "based", "fee", "subject", "wa", "period", "change", "method", "content", "december", "six", "three", "balance", "attributable", "thousand",
              "entity", "result", "price", "data", "party", "vehicle", "contract", "respectively", "basis", "condition", "quarter", "year", "class", "third", "following",
              "date", "us", "statement", "op", "per", "generally", "'s", '', "billion", "ha", "one", "whether", "primary", "thousands", "reit", "property",
              "capital", "could", "lease", "segment", "loan", "service", "due", "tax", "primarily", "convertible"]

tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1)).fit(recent_annual_report["Clean"])
train_vectors = tfidf.transform( [" ".join(recent_annual_report["Clean"].tolist())] ).toarray()
tfidf_df = pd.DataFrame(train_vectors, columns=tfidf.get_feature_names())
critical_words = tfidf_df.T.sort_values(0, ascending=False)[:10].index.tolist()

recent_annual_report["SentTFIDF"] = pd.DataFrame(tfidf.transform(recent_annual_report["Clean"]).toarray(), index=recent_annual_report.index).sum(axis=1)

C:\Users\RUOYUC~1\AppData\Local\Temp/ipykernel_8768/185188835.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_annual_report["SentTFIDF"] = pd.DataFrame(tfidf.transform(recent_annual_report["Clean"]).toarray(), index=recent_annual_report.index).sum(axis=1)


In [28]:
recent_annual_report

,Dirty,Clean,Year,Period,Type,Ticker,SentTFIDF
0,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,united state security exchange commission wash...,2021,Q2,10Q,ZG,6.253939
1,Yes No Indicate by check mark whether the regi...,yes indicate check mark whether registrant ha ...,2021,Q2,10Q,ZG,4.428124
2,Yes No Indicate by check mark whether the regi...,yes indicate check mark whether registrant lar...,2021,Q2,10Q,ZG,3.200138
3,See the definitions of “large accelerated file...,see definition “large accelerated filer” “acce...,2021,Q2,10Q,ZG,3.200067
4,"If an emerging growth company, indicate by che...",emerging growth company indicate check mark re...,2021,Q2,10Q,ZG,4.431594
...,...,...,...,...,...,...,...
712,Section 1350 as adopted pursuant to Section 90...,section adopted pursuant section sarbanesoxley...,2021,Q2,10Q,ZG,2.105100
713,Certification of Chief Financial Officer Pursu...,certification chief financial officer pursuant...,2021,Q2,10Q,ZG,2.220736
714,Section 1350 as adopted pursuant to Section 90...,section adopted pursuant section sarbanesoxley...,2021,Q2,10Q,ZG,2.105100
715,60 Table of Contents SIGNATURES Pursuant to th...,table content signature pursuant requirement s...,2021,Q2,10Q,ZG,3.640732


In [27]:
critical_words

['home',
 'increase',
 'zillow',
 'revenue',
 'expense',
 'loss',
 'mortgage',
 'sale',
 'condensed',
 'offer']

# Identify key sentences and their respective PY counterparts

In [36]:
res = recent_annual_report\
    .query("Clean.str.contains(' | '.join(@critical_words))", engine="python")\
    .query("~Clean.str.contains('table content', regex=False)", engine="python")\
    .query("Dirty.str.contains('\$', regex=True)", engine="python")\
    .query("Dirty.str.len() < @words_in_sentence_threshold", engine="python")

In [37]:
res["SimilarSentencePriorYear"] = res["Dirty"].apply(
    lambda sentence: cosinesimilarity.CompareStringWithOptions(
        sentence, 
        last_annual_report\
        .query("Dirty.str.contains('\$', regex=True)", engine="python")\
        .query("Dirty.str.len() < @words_in_sentence_threshold", engine="python")\
        .Dirty
    )
)

In [38]:
res

,Dirty,Clean,Year,Period,Type,Ticker,SentTFIDF,SimilarSentencePriorYear
195,We have accumulated state tax losses of approx...,accumulated state tax loss approximately mill...,2021,Q2,10Q,ZG,2.203863,(We have accumulated state tax losses of appro...
231,"As of June 30, 2021, the value of homes under ...",june value home contract closed wa billion,2021,Q2,10Q,ZG,1.342669,"[(For the six months ended June 30, 2020, net ..."
335,The Company anticipates gross proceeds from th...,company anticipates gross proceeds sale note a...,2021,Q2,10Q,ZG,2.177360,[(We have accumulated state tax losses of appr...
574,We have accumulated state tax losses of approx...,accumulated state tax loss approximately mill...,2021,Q2,10Q,ZG,2.203863,(We have accumulated state tax losses of appro...


In [35]:
res.to_clipboard()